In [ ]:
# default_exp tutorial

# Installing on Raspbian 64 bit

> A tutorial on installing the `openhsi` library and camera SDKs on a Raspberry Pi 4 with 8 GB RAM.

> Warning: This tutorial is a work in progress. 

> Warning: Using `openhsi` requires generating large datacubes which take up lots of RAM. It is recommended to get a Raspberry Pi 4 with as much RAM as possible, especially if you want to capture raw data. Otherwise, you will need to a `processing_lvl` that includes binning to reduce the datacube size. 

A Raspberry Pi does not have the same processing power as a typical laptop/computer and so expect the framerate to drop slightly. You want a lightweight operating system so the camera can run as fast as possible - this rules out the Ubuntu 64 bit OS. If you have 8 GB of RAM, definitely get the Raspbian 64 bit OS so you can use > 3 GB per process. Another benefit of running on ARM64 bit is that there are prebuilt OpenCV Python wheels - if your project requires `opencv-python`, then 64 bit is needed.

> Note: The latest 64 bit Raspbian is based on Debian 11 Bullseye (equivalent to Ubuntu 21+) and so some packages may not be updated to this platform yet. Also `systemd` is used to run scripts on startup. 

**Step 1**: Download Raspbian 64 bit OS
**Step 2**: Flash a high speed micro SD card. The full install will be about 8 GB so use a larger capacity card if you can. 

> Tip: Hyperspectral datacubes are really large so best to save them to an external SSD.

**Step 3**: Install `vim` or your favourite command line editor
`sudo apt-get install vim`

**Step 4**: Install your Python version of choice.
The default system Python should be 3.9.3. However, if you want to use `FlirCamera`, the manufacturer SDK and Python library requires Python <= 3.8. 

To install any version of Python, you can use `pyenv`
```bash
sudo apt-get install openssl
sudo apt-get install --no-install-recommends make build-essential libssl-dev zlib1g-dev libbz2-dev libreadline-dev libsqlite3-dev wget curl llvm libncurses5-dev xz-utils tk-dev libxml2-dev libxmlsec1-dev libffi-dev liblzma-dev
sudo apt-get install -y gcc make zlib1g-dev libreadline-dev libreadline8 sqlite3 libsqlite3-dev libbz2-dev python-tk python3-tk tk-dev
curl https://pyenv.run | bash

export PATH="$HOME/.pyenv/bin:$PATH"
pyenv install 3.8.12
export PATH="$HOME/.pyenv/bin:$HOME/.pyenv/shims:$PATH"


```

**Step 4.1**: Install NodeJS
This is needed to use the Bokeh plotting library and run interactive tools in a web browser.
`sudo apt install nodejs`

**Step 5**: Enable I2C and UART
Open the Raspberry Pi config tool.
`sudo raspi-config`
And enable I2C and UART. For more instructions see [https://askubuntu.com/questions/1273700/enable-spi-and-i2c-on-ubuntu-20-04-raspberry-pi](https://askubuntu.com/questions/1273700/enable-spi-and-i2c-on-ubuntu-20-04-raspberry-pi).
After enabling them, add the user pi to the group.
`sudo usermod -a -G dialout pi`
`sudo usermod -a -G i2c pi`

Now, to use UART at high speed, we need to change a few more settings documented here
https://www.abelectronics.co.uk/kb/article/1035/serial-port-setup-in-raspberry-pi-os

**Step 5.1**: Disable serial login shell
remove `console=serial0,115200` from /boot/cmdline.txt
then reboot.
**Step 5.2**: Use PI0UART instead of miniUART
Add to end of file /boot/config.txt
`dtoverlay=disable-bt`
Then in the terminal
`sudo systemctl disable hciuart`
`sudo reboot`


## Installing the Battery Hat software

To deploy the Raspberry Pi on a remote platform, it needs power. We use the X728 version 2.1 battery hat from Suptronics which lets us turn on and safe shutdown the Pi while indicating power left. 

Since we are using Debian 11, we need to use systemd to enable /etc/rc-local which the battery hat scripts use. Follow the steps in 
https://www.linuxbabe.com/linux-server/how-to-enable-etcrc-local-with-systemd

The software for the battery hat can be found here:
https://wiki.geekworm.com/X728-Software
Be aware that you need to make some changes to get them to install
Change `python-smbus` to `python3-smbus`
`sudo apt-get install python3-smbus`
`pip install smbus`
`pip install RPi.GPIO`
Python 2 is deprecated so convert x728pld.py to Python 3 syntax.
Add PY_VERSION=3 on line 86 in x728-v2.0.sh (or equivalent in v2.1)


## Installing the FLIR SDK

You will need to install Spinnaker however you'll find that `spinview-qt` will not install because it is missing a dependency which we actually don't need (unless you want to use the GUI called SpinView).
`sudo apt install qtbase5-dev qtchooser qt5-qmake qtbase5-dev-tools`
Follow the hacks found in
https://askubuntu.com/questions/1335184/qt5-default-not-in-ubuntu-21-04
I extracted the offending package, changed a line stating the dependency, then repackaged the files. 

More information on getting the FLIR SDK on a Raspberry Pi and Jetson is here
https://www.flir.com.au/support-center/iis/machine-vision/application-note/using-spinnaker-on-arm-and-embedded-systems/

> Tip: Get the aarch64 and arm64 packages (no amd)

The install then becomes just
`sudo sh install_[according to your version].sh`
`pip install [according to your Python version].whl file`


## Mounting SSD without sudo

The reason why we want to mount the SSD without sudo is that we can then write to it without sudo.

First, identify your SSD using
`sudo fdisk -l`
I followed the steps in 
https://blog.onetwentyseven001.com/mounting-without-sudo/index.html

Congratulations, you now have a Raspbian 64 bit OS that can run the `FlirCamera` and `openhsi`!